## Imports

In [17]:
import pandas as pd 
import numpy as np 
import requests
import xlsxwriter
import math

## List of S&P 500 Stocks

In [18]:
stocks = pd.read_csv('sp_500_static.csv')

## API Token (Sandbox Mode)

In [19]:
from secret import ALPHA_VANTAGE_API_KEY

## API Call

In [20]:
cols = ['Ticker', 'Price', 'Market Cap', 'Num of Shares to Buy']
trade_df = pd.DataFrame(columns=cols)
NUM_OF_STOCKS = 2

# Free API Key doesn't allow enough calls
for symbol in stocks['Ticker'][:NUM_OF_STOCKS]:

    api_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}'
    data = requests.get(api_url).json()
    market_cap = float(data['MarketCapitalization'])
    
    api_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}'
    data = requests.get(api_url).json()
    price = float(data['Global Quote']['05. price'])


    trade_df.loc[len(trade_df)] = {'Ticker' : symbol, 'Price' : price, 'Market Cap' : market_cap, 'Num of Shares to Buy' : 0}

trade_df

,Ticker,Price,Market Cap,Num of Shares to Buy
0,A,111.41,3.259712e+10,0
1,AAL,12.90,8.428369e+09,0


## Batch Calls

In [21]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [22]:
symbol_groups = list(chunks(stocks['Ticker'], 5))

## Num of Shares

In [25]:
trade_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Ticker                2 non-null      object 
 1   Price                 2 non-null      float64
 2   Market Cap            2 non-null      float64
 3   Num of Shares to Buy  2 non-null      int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 80.0+ bytes


In [27]:
PORTFOLIO_VALUE = 1e7
allocation = float(PORTFOLIO_VALUE) / len(trade_df) if len(trade_df) else 0
trade_df['Num of Shares to Buy'] = trade_df['Price'] * allocation

## Output

In [44]:
trade_df.to_excel('output.xlsx', index=False)